In [2]:
import pandas as pd
import numpy as np
import math
data=pd.read_csv('IRIS.csv')
data=np.array(data)
np.random.shuffle(data)

In [3]:
n_features=len(data[0])-1
n_data=len(data)

In [4]:
#Iris-setosa
for i in range(n_data):
    if(data[i][n_features]=="Iris-setosa"):
        data[i][n_features]=1
    else:
        data[i][n_features]=0

In [ ]:
thresold=0.6
n_epochs=10000
learning_rate=0.3
k_fold=10
layer=[n_features,5,2]
# weights=[[[1/(layer[i]*layer[i+1])]*layer[i] for j in range(layer[i+1])] for i in range(len(layer)-1)]
weights=[[[np.random.random_sample()]*layer[i] for j in range(layer[i+1])] for i in range(len(layer)-1)]
bias=[[1/layer[i+1]]*layer[i+1] for i in range(len(layer)-1)]
nodes=[[0]*layer[i] for i in range(len(layer))]
error=[[0]*layer[i] for i in range(len(layer))]
# print(nodes)
# print(weights)
#kfold variables
width=int(n_data/k_fold)
k_fold_index=[]
start=0
for i in range(k_fold-1):
    k_fold_index.append([start,start+width])
    start=start+width
k_fold_index.append([start,n_data])


def kfolddata(j,data):
    x_test=data[k_fold_index[j][0]:k_fold_index[j][1],0:n_features]
    y_test=data[k_fold_index[j][0]:k_fold_index[j][1],n_features]
    if(j+1<=9):
        x_train=data[k_fold_index[j][1]:,0:n_features]
        y_train=data[k_fold_index[j][1]:,n_features]
        if(j-1!=-1):
            x_train=np.append(x_train,data[0:k_fold_index[j][0],0:n_features],axis=0)
            y_train=np.append(y_train,data[0:k_fold_index[j][0],n_features],axis=0)
    else:
        x_train=data[0:k_fold_index[j][0],0:n_features]
        y_train=data[0:k_fold_index[j][0],n_features]
    return x_train,y_train,x_test,y_test
    
def function(row,nodes,weights,bias,thresold,layer):
    for i in range(len(row)):
        nodes[0][i]=row[i]
    for lay in range(1,len(layer)):
        for j in range(layer[lay]):
            y=bias[lay-1][j]
            for prevlayer,k in enumerate(weights[lay-1][j]):
                y=y+nodes[lay-1][prevlayer]*k
            nodes[lay][j]=1/(1+math.exp(-y))
#     print(nodes[len(layer)-1])
    if(nodes[len(layer)-1][0]>nodes[len(layer)-1][1]):
        y=1
    else:
        y=0
    return nodes,y

def back_prop(weights,nodes,thresold,bias,learning_rate,cost,error,z):
    #output layer error
    if z==1:
        z1=1
        z2=0
    else:
        z1=0
        z2=1
    error[len(error)-1][0]=(z1-nodes[len(error)-1][0])*(1-nodes[len(error)-1][0])*nodes[len(error)-1][0]
    error[len(error)-1][0]=(z2-nodes[len(error)-1][0])*(1-nodes[len(error)-1][0])*nodes[len(error)-1][0]
#     print(error[len(error)-1][0])
    #hidden layers
    for lay in range(len(error)-2,0,-1):
        for nod in range(len(nodes[lay])):
            err=0
            for nxtlaynod in range(len(nodes[lay+1])):
                err+=error[lay+1][nxtlaynod]*weights[lay][nxtlaynod][nod]             
            error[lay][nod]=nodes[lay][nod]*(1-nodes[lay][nod])*err*-1
    #update weights
    for lay in range(len(weights)):
        for j in range(len(weights[lay])):
            for i in range(len(weights[lay][j])):
                weights[lay][j][i]+=learning_rate*nodes[lay][i]*error[lay+1][j]
    for lay in range(len(bias)):
        for i in range(len(bias[lay])):
            bias[lay][i]+=learning_rate*error[lay+1][i]
    return weights,bias

from valid import validation
for epoch in range(n_epochs):
#     print("epoch no",epoch+1,"+"*60)
    foldres=validation.foldwise()
    for j in range(k_fold):
#         print("fold no:",j)
        foldres.reset()
        x_train,y_train,x_test,y_test=kfolddata(j,data)
        for i in range(len(x_train)):
            nodes,y=function(x_train[i],nodes,weights,bias,thresold,layer)
            z=y_train[i]
            cost=z-y
            foldres.valid(y,cost,0)
            weights,bias=back_prop(weights,nodes,thresold,bias,learning_rate,cost,error,z)
#             print(weights)
        for i in range(len(x_test)):
            nodes,y=function(x_test[i],nodes,weights,bias,thresold,layer)
            z=y_test[i]
            cost=z-y
            foldres.valid(y,cost,1)
#         foldres.printfoldresult()
        foldres.averageresults()
    if (epoch%100)==0:
        print("epoch no",epoch+1,"+"*60)
        foldres.printepochresult()
        print(weights)


epoch no 1 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.5011055432717415
train Precision +: 0.0545404963185165
train Precision -: 0.500317854540403
train recall +: 0.012765685836471566
train recall -: 0.9883501197216755
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[0.9168892237692134, 0.9169177552960847, 0.9169606529421849, 0.9169795822483319], [0.4502276691166848, 0.43101904454930945, 0.40520109154048584, 0.390860351779072], [0.4650415032727574, 0.40391849733906043, 0.11200342923561964, 0.08543083519558568], [0.859598850527109, 0.8596440185807271, 0.8597203490915353, 0.8597495930620185], [0.4720498062237303, 0.4058893257406657, 0.22892995149081713, 0.18732413759214084]], [[-0.21980630608795143, 

epoch no 801 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.2314349544732264, 1.1263568526508303, 1.0074878345095015, 0.9311025630495811], [1.3337387468628397, 1.0512245646352483, 0.6076584134945073, 0.4083935174962454], [1.364082775999693, 1.0838742321232808, 0.20479912401010913, 0.05759172353552732], [1.2203219012343365, 1.100663638040713, 0.9624531880374176, 0.8754127067770044], [1.3696689328384306, 1.0917644883529614, 0.3235850704251417, 0.15906197081069962]], [[0.14977782403419468, 0.010671272413542932, 0.09715404353

epoch no 1601 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.3419924527679774, 1.1999714071529037, 1.038516932050937, 0.9355791197779685], [1.421858171123384, 1.1110015563532571, 0.6304525083055231, 0.41119155739104113], [1.4795145432610075, 1.17093370787117, 0.21658115100699343, 0.05371862295766589], [1.3370938416485856, 1.1785447282555783, 0.9950899795369277, 0.8800905649294828], [1.4555331799790605, 1.152532088745222, 0.34044273682872855, 0.15956764187787012]], [[0.14962746599416002, 0.03895509442314904, 0.112153868245

epoch no 2401 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.4020033353863817, 1.2398266207490736, 1.0553961660440858, 0.9380258679417308], [1.4806445102629329, 1.1507858638026092, 0.6457057709559709, 0.4130728544287323], [1.5434318919226, 1.21866311793523, 0.22382246028026678, 0.05187260922481173], [1.4005078353927483, 1.220722094003527, 1.0128581450531686, 0.8826499087890597], [1.5124551157231145, 1.1926786982087991, 0.35175987405614006, 0.15995784473232522]], [[0.14831516753505022, 0.05761825709192835, 0.11811677474820

epoch no 3201 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.4434297411532007, 1.267297348824844, 1.0670616848993901, 0.9397216111533555], [1.52434771802036, 1.180307587941044, 0.6570797651047305, 0.4144843229569072], [1.587748868436692, 1.2515521093655204, 0.22915068148221465, 0.05072165921427517], [1.4441201892908269, 1.249681728504942, 1.0250948387679095, 0.8844178517961242], [1.555008545822436, 1.2226107530121093, 0.3603090049437022, 0.16028518338799108]], [[0.14756911502908387, 0.07047797432705771, 0.1218679053932877

epoch no 4001 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.4751363642365085, 1.2882996860531142, 1.0759971063020324, 0.9410231006926539], [1.5589306949009265, 1.20363347750008, 0.6661038716721559, 0.41561065766597044], [1.621700984596356, 1.2766366122543786, 0.23340150926408412, 0.04991098766756745], [1.4773841925768585, 1.2717439427120751, 1.0344366871045267, 0.8857705080480361], [1.5888905083178586, 1.246389512750001, 0.36717801744256345, 0.160571224336849]], [[0.14725137048971518, 0.07997419196052499, 0.1246866841072

epoch no 4801 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
average result for  10 fold
Last fold confusion matrix trainTT: 0  trainTF: 44  trainFF: 46  trainFT: 0
Last fold testTT: 0  testTF: 6  testFF: 4  testFT: 0
train_accuracy: 0.4999944445061722
train Precision +: 0.0
train Precision -: 0.4999944445061722
train recall +: 0.0
train recall -: 0.999977756288267
test_accuracy 0.49995000499950004
test Precision +: 0.0
test Precision -: 0.49995000499950004
test recall +: 0.0
test recall -: 0.9997814809172014
[[[1.5008536753166075, 1.3053201460318427, 1.0832488426365339, 0.9420809988210914], [1.58744943946431, 1.2228451102879494, 0.6735622960438842, 0.41654634081195974], [1.6492344456147345, 1.2969079976775373, 0.23695471780458327, 0.049298359281644985], [1.5042849206040894, 1.2895694282456245, 1.0419967183961512, 0.8868670396616268], [1.6169778827152808, 1.266063460588911, 0.37291721845379927, 0.1608269783694838]], [[0.14720455675464944, 0.0873588419753654, 0.12698861815